<a href="https://colab.research.google.com/github/ellenwterry/PoliticalAnalysis/blob/main/Reg_NE_and_LogReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import patsy
from sklearn.linear_model import LogisticRegression


!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

import patsy
from sklearn.linear_model import LogisticRegression

!pip install pystan==3.7.0
#!pip install pystan
!pip install corner
import stan

import plotly.express as px
import plotly.graph_objects as go

!pip install geopy
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
!pip install pygris
# import matplotlib.pyplot as plt
from pygris import core_based_statistical_areas
from pygris import tracts

from google.colab import files


import geopandas as gpd
import folium
# from google.colab import files


Get Data

In [61]:
# ---------- Get Data from Github ---------- #

url = 'https://raw.githubusercontent.com/ellenwterry/PoliticalAnalysis/main/VoteBase.csv'
VoteBase = pd.read_csv(url)

Tidy Data

In [62]:
# ---------- Clean up data ---------- #

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(VoteBase['Sex'])

codes = {'NR':0, 'M': 1, 'F': 2}
VoteBase['Sex'] = VoteBase['Sex'].map(codes)

VoteBase['Age']=VoteBase.Age.astype('int32')

#VoteBase['LastPrimary'] = le.transform(VoteBase['LastPrimary'])
codes = {'NR':0, 'R': 1, 'D':2}
VoteBase['LastPrimary'] = VoteBase['LastPrimary'].map(codes)

#VoteBase['Education'] = le.transform(VoteBase['Education'])
codes = {'NR':0, 'HS': 1, 'Some College':2, 'Bachelor':3, 'Masters':4, 'Doctorate':5}
VoteBase['Education'] = VoteBase['Education'].map(codes)

#VoteBase['HHIncome'] = le.transform(VoteBase['HHIncome'])
codes = {'NR':0, 'Under 50k': 1, '50k-100k':2, '100k-200k':3, '200k-300k':4, '300k-500k':5, 'Over 500k':6}
VoteBase['HHIncome'] = VoteBase['HHIncome'].map(codes)

#VoteBase['ReligiousAffil'] = le.transform(VoteBase['ReligiousAffil'])
codes = {'NR':0,'Protestant': 1, 'Catholic':2, 'Other':3, 'None':4}
VoteBase['ReligiousAffil'] = VoteBase['ReligiousAffil'].map(codes)

#VoteBase['Support24'] = le.transform(VoteBase['Support24'])
codes = {'R':0, 'D': 1}
VoteBase['Support24'] = VoteBase['Support24'].map(codes)
# NOTE: NAs were excluded from sample so that algorithms could score using logistic scale - 2nd pass will use imputed values

#VoteBase['TopIssue'] = le.transform(VoteBase['TopIssue'])
codes = {'NR':0, 'RFree':1, 'Parents':2, 'Crime':3, 'Economy':4, 'Womens':5, 'Education':6, 'Environment':7, 'Democracy':8}
VoteBase['TopIssue'] = VoteBase['TopIssue'].map(codes)

# This is for the second data source (later)
codes = {'NS':0, 'NR':1,'Signed':2}
VoteBase['RRPetition'] = VoteBase['RRPetition'].map(codes)

Split into Training and Test Sets

In [63]:
#np.random.randint(0, 10, size=4)
rows = VoteBase.shape[0]-1
testInd = np.random.randint(0, (VoteBase.shape[0]-1), size=100)
tstBase = VoteBase[VoteBase.index.isin(testInd)]
trainBase = VoteBase[~VoteBase.index.isin(tstBase)]


In [64]:
yTrain = trainBase['Support24']
yTest = tstBase['Support24']
Xmatrix = patsy.dmatrix('Age + Sex + Education + HHIncome+ ReligiousAffil + LastPrimary + TopIssue', trainBase)
tstMatrix = patsy.dmatrix('Age + Sex + Education + HHIncome+ ReligiousAffil + LastPrimary + TopIssue', tstBase)
rows = Xmatrix.shape[0]
columns = Xmatrix.shape[1]
rows, columns

(2498, 8)

Train LogReg Model

In [65]:
model = LogisticRegression(solver='sag')
model.fit(Xmatrix, yTrain)
Pred = model.predict(Xmatrix)
Probs = pd.DataFrame(model.predict_proba(tstMatrix))
theta = np.matrix(model.coef_)
intercept = model.intercept_
Probs[1]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



0     0.179316
1     0.185081
2     0.083378
3     0.358175
4     0.076757
        ...   
93    0.914493
94    0.973976
95    0.929655
96    0.993319
97    0.990596
Name: 1, Length: 98, dtype: float64

Select Test Samples and Confirm using Equation.

In [66]:
def stable_sigmoid(x):
  # Using np.where to avoid numerical overflow or underflow.
  return np.where(x >= 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))

# This logreg algorithm breaks out the bias separately for analysis with multinomials.

Prob2 = stable_sigmoid((np.dot(model.coef_,tstMatrix[0].transpose())+ model.intercept_).transpose())
Prob2

array([0.17931555])

In [67]:
Probs[1]

0     0.179316
1     0.185081
2     0.083378
3     0.358175
4     0.076757
        ...   
93    0.914493
94    0.973976
95    0.929655
96    0.993319
97    0.990596
Name: 1, Length: 98, dtype: float64

In [88]:
VoteBase['Prob'] = pd.DataFrame(model.predict_proba(Xmatrix))[1]

In [89]:
VoteBase

,ID,Name,Sex,Age,LastPrimary,Latitude,Longitude,Education,HHIncome,ReligiousAffil,Support24,TopIssue,RRPetition,Prob
0,1,Voter 1,1,51,0,41.00544,-73.62954,2,3,1,0,0,0,0.362846
1,2,Voter 2,1,52,0,41.04027,-73.63011,2,2,1,0,0,0,0.251569
2,3,Voter 3,2,54,0,41.05734,-73.65044,3,6,1,1,4,0,0.763076
3,4,Voter 4,1,59,0,41.03255,-73.60403,2,3,1,0,0,0,0.330180
4,5,Voter 5,1,36,0,41.06684,-73.57837,2,3,4,0,0,0,0.932369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,2494,Voter 2494,2,62,0,41.07108,-73.63842,4,3,4,1,5,1,0.941773
2494,2495,Voter 2495,2,62,2,41.03758,-73.57932,3,3,0,1,5,1,0.342222
2495,2496,Voter 2496,2,65,2,41.01605,-73.58949,3,6,1,1,0,1,0.786879
2496,2497,Voter 2497,1,67,2,40.99031,-73.65605,0,6,0,1,0,1,0.470675


In [90]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=VoteBase['Prob'], marker_color='#378796', opacity=0.05, name = "Vote Prob"))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.9, nbinsx=100)
fig.update_layout(
    autosize=False,
    width=800,
    height=400,
    plot_bgcolor = "white",
    xaxis=dict(title='Probability of Vote',),
)

fig.show()

In [91]:
theta

matrix([[-1.13240003, -0.0180427 , -0.40193795,  0.27249577,  0.50918632,
          0.97201492,  0.36186393,  0.09716475]])

In [92]:
color_map = {
    1: "#5218fa"
}

fig2 = go.Figure()
fig2.add_trace(go.Histogram(x=VoteBase['Prob'], marker_color=VoteBase["TopIssue"].map(color_map)))
fig2.update_traces(opacity=0.9, nbinsx=100)
fig2.update_layout(
    plot_bgcolor = "white",
    xaxis=dict(title='Probability of Vote',),
)

fig2.show()

